# Data Cleaning

## Crashes Dataset

In [1]:
import pandas as pd
import numpy as np

crashes = pd.read_csv('../data/crashes_crashes.csv')

In [2]:
crashes['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 310493
FAILING TO YIELD RIGHT-OF-WAY                                                        87668
FOLLOWING TOO CLOSELY                                                                77957
NOT APPLICABLE                                                                       42294
IMPROPER OVERTAKING/PASSING                                                          39303
FAILING TO REDUCE SPEED TO AVOID CRASH                                               33776
IMPROPER BACKING                                                                     31565
IMPROPER LANE USAGE                                                                  28699
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  26639
IMPROPER TURNING/NO SIGNAL                                                           26520
DISREGARDING TRAFFIC SIGNALS                                      

### Getting rid of rows without target variable

In [3]:
crashes_prime_cause = crashes[(crashes['PRIM_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE') & (crashes['PRIM_CONTRIBUTORY_CAUSE'] != 'NOT APPLICABLE')]

### Binning Target Variable

In [4]:
breaking_laws_list = ['DISREGARDING TRAFFIC SIGNALS', 'DISREGARDING STOP SIGN', 'DISREGARDING ROAD MARKINGS', 
                  'DISREGARDING OTHER TRAFFIC SIGNS', 'DISREGARDING YIELD SIGN', 'FAILING TO YIELD RIGHT-OF-WAY'] 

bad_driving_list = ['DRIVING ON WRONG SIDE/WRONG WAY', 'FOLLOWING TOO CLOSELY', 'IMPROPER OVERTAKING/PASSING', 
                    'FAILING TO REDUCE SPEED TO AVOID CRASH', 'TURNING RIGHT ON RED','EXCEEDING SAFE SPEED FOR CONDITIONS',
                    'EXCEEDING AUTHORIZED SPEED LIMIT', 'IMPROPER LANE USAGE', 'PHYSICAL CONDITION OF DRIVER', 
                  'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE','IMPROPER BACKING', 'IMPROPER TURNING/NO SIGNAL']

distraction_list = ['TEXTING', 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)', 
                    'DISTRACTION - FROM INSIDE VEHICLE','CELL PHONE USE OTHER THAN TEXTING']

drinking_list = ['OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER', 
                 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)', 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)']

road_list = ['DISTRACTION - FROM OUTSIDE VEHICLE', 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS', 'ROAD CONSTRUCTION/MAINTENANCE', 'EQUIPMENT - VEHICLE CONDITION', 
             'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 'WEATHER']

other_list= ['PASSING STOPPED SCHOOL BUS', 'OBSTRUCTED CROSSWALKS', 'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
             'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT', 'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST', 'ANIMAL', 'TURNING RIGHT ON RED', 
             'RELATED TO BUS STOP'] 

binning_list = [breaking_laws_list, bad_driving_list, distraction_list, drinking_list, road_list, other_list]
value_list = ['BREAKING LAW', 'BAD DRIVING', 'DISTRACTION INSIDE VEHICLE', 'DRINKING/DRUGS', 'OUTSIDE FACTORS', 'OTHER']



for group, value in zip(binning_list, value_list):
    crashes_prime_cause['PRIM_CONTRIBUTORY_CAUSE'] = crashes_prime_cause['PRIM_CONTRIBUTORY_CAUSE'].replace(to_replace = group, value = value)

crashes_prime_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

/var/folders/bt/mf607s9n0xs6sr08g8c44w9c0000gn/T/ipykernel_2415/325155699.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_prime_cause['PRIM_CONTRIBUTORY_CAUSE'] = crashes_prime_cause['PRIM_CONTRIBUTORY_CAUSE'].replace(to_replace = group, value = value)


PRIM_CONTRIBUTORY_CAUSE
BAD DRIVING                   277735
BREAKING LAW                  114860
OUTSIDE FACTORS                28657
DRINKING/DRUGS                 14797
DISTRACTION INSIDE VEHICLE      7261
OTHER                           2809
Name: count, dtype: int64

### Cleaning NANs

In [5]:
crashes_prime_cause_filled = crashes_prime_cause.fillna({'INTERSECTION_RELATED_I': 'N', 'NOT_RIGHT_OF_WAY_I': 
                                                         'N', 'HIT_AND_RUN_I':'N'})

crashes_prime_cause_filled = crashes_prime_cause_filled.dropna(subset=['LATITUDE', 'LONGITUDE', 'INJURIES_TOTAL', 'INJURIES_FATAL', 
                                                                           'MOST_SEVERE_INJURY'])


### Creating 30 'neighborhoods' by KNN

In [6]:
from sklearn.cluster import KMeans

n_clusters = 30  # Number of clusters to create
X = crashes_prime_cause_filled[['LONGITUDE', 'LATITUDE']]

# Create a K-Means clustering model
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X)

# Add cluster labels to your data
cluster_labels = kmeans.labels_
crashes_prime_cause_filled['GEO_KMEANS_Cluster'] = cluster_labels


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


### Dropping Useless Columns

In [7]:
crashes_prime_cause_filled = crashes_prime_cause_filled.drop(columns = ['CRASH_DATE_EST_I', 'DEVICE_CONDITION', 'REPORT_TYPE', 'DATE_POLICE_NOTIFIED',
                                                          'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME','BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 
                                                          'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 
                                                          'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_NO_INDICATION', 
                                                          'INJURIES_UNKNOWN', 'LOCATION', 'LANE_CNT', 'CRASH_DATE', 
                                                          'INJURIES_REPORTED_NOT_EVIDENT', 'TRAFFIC_CONTROL_DEVICE', 'INJURIES_TOTAL', 
                                                          'INJURIES_FATAL'])

### Transforming Categorical Data to Numerical Data

In [9]:
crashes_cleaned = pd.get_dummies(crashes_prime_cause_filled, columns = ['WEATHER_CONDITION', 'LIGHTING_CONDITION','FIRST_CRASH_TYPE',
                                                                            'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 
                                                                            'SEC_CONTRIBUTORY_CAUSE', 'ROAD_DEFECT','MOST_SEVERE_INJURY'],
                                                                             drop_first=True, dtype=int)
                                                                                                 




#binary
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['CRASH_TYPE','INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE']:
    crashes_cleaned[col] = le.fit_transform(crashes_cleaned[col])
    


In [10]:
crashes_cleaned.to_csv('../data/crashes_cleaned.csv', index=True)

## Binning Feature Classes

In [11]:
crashes_cleaned_binned = crashes_prime_cause_filled

In [12]:
precipitation_list = ['RAIN', 'SNOW', 'FREEZING RAIN/DRIZZLE', 'SLEET/HAIL', 'BLOWING SNOW']

wind_list = ['SEVERE CROSS WIND GATE', 'BLOWING SAND, SOIL, DIRT']

visibility_list = ['CLOUDY/OVERCAST', 'FOG/SMOKE/HAZE', 'OTHER']

binning_list = [precipitation_list, wind_list, visibility_list]
value_list = ['PRECIPITATION', 'WINDY', 'VISIBILITY']


for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['WEATHER_CONDITION'] = crashes_cleaned_binned['WEATHER_CONDITION'].replace(to_replace = group, value = value)

crashes_cleaned_binned['WEATHER_CONDITION'].value_counts()


WEATHER_CONDITION
CLEAR            353868
PRECIPITATION     61532
VISIBILITY        16798
UNKNOWN           10089
WINDY                95
Name: count, dtype: int64

In [13]:
crashes_prime_cause_filled['LIGHTING_CONDITION'].value_counts()

twighlight_list = ['DUSK', 'DAWN']

binning_list = [twighlight_list]
value_list = ['TWIGHLIGHT']


for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['LIGHTING_CONDITION'] = crashes_cleaned_binned['LIGHTING_CONDITION'].replace(to_replace = group, value = value)
crashes_cleaned_binned['LIGHTING_CONDITION'].value_counts()

LIGHTING_CONDITION
DAYLIGHT                  294529
DARKNESS, LIGHTED ROAD    101566
TWIGHLIGHT                 20547
DARKNESS                   19127
UNKNOWN                     6613
Name: count, dtype: int64

In [14]:
crashes_prime_cause_filled['FIRST_CRASH_TYPE'].value_counts()

non_vehicle_list = ['FIXED OBJECT', 'PEDESTRIAN', 'OTHER OBJECT', 'ANIMAL']

non_car_vehicle = ['PEDALCYCLIST', 'TRAIN']

driving_same_direction = ['REAR END', 'SIDESWIPE SAME DIRECTION']

opposite_direction = ['SIDESWIPE OPPOSITE DIRECTION', 'HEAD ON']

backing_up = ['REAR TO FRONT', 'REAR TO REAR', 'REAR TO SIDE']

binning_list = [non_vehicle_list, non_car_vehicle, driving_same_direction, opposite_direction, backing_up]
value_list = ['NON_VEHICLE', 'NON_CAR', 'DRIVING_SAME_DIRECTION', 'DRIVING_OPP_DIRECTION', 'BACKING_UP']


for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['FIRST_CRASH_TYPE'] = crashes_cleaned_binned['FIRST_CRASH_TYPE'].replace(to_replace = group, value = value)

crashes_cleaned_binned['FIRST_CRASH_TYPE'].value_counts()

FIRST_CRASH_TYPE
DRIVING_SAME_DIRECTION    184670
TURNING                    78654
PARKED MOTOR VEHICLE       62534
ANGLE                      58428
NON_VEHICLE                31161
DRIVING_OPP_DIRECTION      10511
BACKING_UP                  8336
NON_CAR                     6771
OTHER NONCOLLISION          1092
OVERTURNED                   225
Name: count, dtype: int64

In [15]:
crashes_prime_cause_filled['TRAFFICWAY_TYPE'].value_counts()

intersection_list = ['T-INTERSECTION', 'UNKNOWN INTERSECTION TYPE', 'Y-INTERSECTION', 
                     'FIVE POINT, OR MORE', 'ROUNDABOUT', 'L-INTERSECTION', 'FOUR WAY']

unknown_list = ['UNKNOWN', 'NOT REPORTED']

slow_areas = ['PARKING LOT', 'DRIVEWAY']

highway_list = ['DIVIDED - W/MEDIAN (NOT RAISED)', 'DIVIDED - W/MEDIAN BARRIER', 'RAMP']

binning_list = [intersection_list, unknown_list, slow_areas, highway_list]
value_list = ['INTERSECTION', 'UNKNOWN', 'SLOW_AREAS', 'HIGHWAY']


for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['TRAFFICWAY_TYPE'] = crashes_cleaned_binned['TRAFFICWAY_TYPE'].replace(to_replace = group, value = value)
crashes_cleaned_binned['TRAFFICWAY_TYPE'].value_counts()


TRAFFICWAY_TYPE
NOT DIVIDED         192509
HIGHWAY             109824
ONE-WAY              46733
INTERSECTION         43640
SLOW_AREAS           24705
OTHER                11269
ALLEY                 6274
CENTER TURN LANE      4053
UNKNOWN               2740
TRAFFIC ROUTE          635
Name: count, dtype: int64

In [16]:
crashes_prime_cause_filled['ALIGNMENT'].value_counts()

straight_list= ['STRAIGHT AND LEVEL', 'STRAIGHT ON GRADE', 'STRAIGHT ON HILLCREST']

curve_list = ['CURVE, LEVEL', 'CURVE ON GRADE', 'CURVE ON HILLCREST']

binning_list = [straight_list, curve_list]
value_list = ['STRAIGHT', 'CURVE']


for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['ALIGNMENT'] = crashes_cleaned_binned['ALIGNMENT'].replace(to_replace = group, value = value)

crashes_cleaned_binned['ALIGNMENT'].value_counts()

ALIGNMENT
STRAIGHT    437878
CURVE         4504
Name: count, dtype: int64

In [17]:
crashes_prime_cause_filled['ROADWAY_SURFACE_COND'].value_counts()

wet = ['WET', 'SNOW OR SLUSH', 'ICE', 'SAND, MUD, DIRT']

unknown = ['UNKNOWN', 'OTHER']

binning_list = [wet, unknown]
value_list = ['WET', 'UNKNOWN']


for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['ROADWAY_SURFACE_COND'] = crashes_cleaned_binned['ROADWAY_SURFACE_COND'].replace(to_replace = group, value = value)

crashes_cleaned_binned['ROADWAY_SURFACE_COND'].value_counts()


ROADWAY_SURFACE_COND
DRY        335639
WET         85061
UNKNOWN     21682
Name: count, dtype: int64

In [18]:
crashes_prime_cause_filled['SEC_CONTRIBUTORY_CAUSE'].value_counts()

breaking_laws_list = ['DISREGARDING TRAFFIC SIGNALS', 'DISREGARDING STOP SIGN', 'DISREGARDING ROAD MARKINGS', 
                  'DISREGARDING OTHER TRAFFIC SIGNS', 'DISREGARDING YIELD SIGN', 'FAILING TO YIELD RIGHT-OF-WAY'] 

bad_driving_list = ['DRIVING ON WRONG SIDE/WRONG WAY', 'FOLLOWING TOO CLOSELY', 'IMPROPER OVERTAKING/PASSING', 
                    'FAILING TO REDUCE SPEED TO AVOID CRASH', 'TURNING RIGHT ON RED','EXCEEDING SAFE SPEED FOR CONDITIONS',
                    'EXCEEDING AUTHORIZED SPEED LIMIT', 'IMPROPER LANE USAGE', 'PHYSICAL CONDITION OF DRIVER', 
                  'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE','IMPROPER BACKING', 'IMPROPER TURNING/NO SIGNAL']

distraction_list = ['TEXTING', 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)', 
                    'DISTRACTION - FROM INSIDE VEHICLE','CELL PHONE USE OTHER THAN TEXTING']

drinking_list = ['OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER', 
                 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)', 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)']

road_list = ['DISTRACTION - FROM OUTSIDE VEHICLE', 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS', 'ROAD CONSTRUCTION/MAINTENANCE', 'EQUIPMENT - VEHICLE CONDITION', 
             'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 'WEATHER']

other_list= ['PASSING STOPPED SCHOOL BUS', 'OBSTRUCTED CROSSWALKS', 'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
             'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT', 'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST', 'ANIMAL', 'TURNING RIGHT ON RED', 
             'RELATED TO BUS STOP'] 

binning_list = [breaking_laws_list, bad_driving_list, distraction_list, drinking_list, road_list, other_list]
value_list = ['BREAKING LAW', 'BAD DRIVING', 'DISTRACTION INSIDE VEHICLE', 'DRINKING/DRUGS', 'OUTSIDE FACTORS', 'OTHER']

for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['SEC_CONTRIBUTORY_CAUSE'] = crashes_cleaned_binned['SEC_CONTRIBUTORY_CAUSE'].replace(to_replace = group, value = value)

crashes_cleaned_binned['SEC_CONTRIBUTORY_CAUSE'].value_counts()


SEC_CONTRIBUTORY_CAUSE
NOT APPLICABLE                167856
BAD DRIVING                   114330
UNABLE TO DETERMINE           102924
BREAKING LAW                   31219
OUTSIDE FACTORS                14900
DRINKING/DRUGS                  6652
DISTRACTION INSIDE VEHICLE      3215
OTHER                           1286
Name: count, dtype: int64

In [20]:
crashes_prime_cause_filled['ROAD_DEFECT'].value_counts()

unknown = ['OTHER', 'UNKNOWN']

road_cond = ['RUT, HOLES', 'WORN SURFACE']

binning_list = [unknown, road_cond]
value_list = ['UNKNOWN', 'BAD_ROAD']

for group, value in zip(binning_list, value_list):
    crashes_cleaned_binned['ROAD_DEFECT'] = crashes_cleaned_binned['ROAD_DEFECT'].replace(to_replace = group, value = value)

crashes_cleaned_binned['ROAD_DEFECT'].value_counts()

ROAD_DEFECT
NO DEFECTS           374293
UNKNOWN               61808
BAD_ROAD               5030
SHOULDER DEFECT         874
DEBRIS ON ROADWAY       377
Name: count, dtype: int64

#### Getting Dummy Variables

In [21]:
crashes_cleaned_binned = pd.get_dummies(crashes_cleaned_binned, columns = ['WEATHER_CONDITION', 'LIGHTING_CONDITION','FIRST_CRASH_TYPE',
                                                                            'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 
                                                                            'SEC_CONTRIBUTORY_CAUSE', 'ROAD_DEFECT','MOST_SEVERE_INJURY'],
                                                                             drop_first=True, dtype=int)



#### Label Encoding 

In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['CRASH_TYPE','INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE']:
    crashes_cleaned_binned[col] = le.fit_transform(crashes_cleaned_binned[col])

crashes_cleaned_binned = crashes_cleaned_binned.select_dtypes(include=['number'])
crashes_cleaned_binned = crashes_cleaned_binned.dropna()
crashes_cleaned_binned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 442382 entries, 2 to 798905
Data columns (total 58 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   POSTED_SPEED_LIMIT                                 442382 non-null  int64  
 1   CRASH_TYPE                                         442382 non-null  int64  
 2   INTERSECTION_RELATED_I                             442382 non-null  int64  
 3   NOT_RIGHT_OF_WAY_I                                 442382 non-null  int64  
 4   HIT_AND_RUN_I                                      442382 non-null  int64  
 5   DAMAGE                                             442382 non-null  int64  
 6   PRIM_CONTRIBUTORY_CAUSE                            442382 non-null  int64  
 7   NUM_UNITS                                          442382 non-null  int64  
 8   CRASH_HOUR                                         442382 non-null  int64  
 9 

In [25]:
crashes_cleaned_binned.to_csv('../data/crashes_cleaned_binned.csv', index=True)

## People Dataset

In [ ]:
people = pd.read_csv('../data/crashes_people.csv', low_memory = False)

### Limiting down people to just drivers and keeping relevant columns

In [ ]:
people.drop(people[people['PERSON_TYPE'] != 'DRIVER'].index, inplace=True)

people = people[['CRASH_RECORD_ID', 'SEX', 'AGE', 'PHYSICAL_CONDITION','DRIVER_ACTION', 'DRIVER_VISION']]

### Dealing with weird values/NANS

In [ ]:
people.drop(people[people['AGE'] < 0].index, inplace=True)

people.SEX.fillna('X', inplace=True)

### Categorical -> Numeric

In [1]:
# Get numerical values for categorical columns using OneHotEncoder
cat = ['SEX', 'PHYSICAL_CONDITION', 'DRIVER_ACTION', 'DRIVER_VISION']

# Import OneHotEncoder from Scikit Learn Preprocessing
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop = 'first', sparse = False)

encoded_df = pd.DataFrame(ohe.fit_transform(people[cat]), columns=ohe.get_feature_names(cat))

# Reset the index of both DataFrames
people.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Concatenate the encoded DataFrame with the original DataFrame
people_cleaned = pd.concat([people, encoded_df], axis=1)
people_cleaned.drop(cat, axis = 1, inplace = True)
people_cleaned

NameError: name 'pd' is not defined

# Vehicle Dataset

In [ ]:
vehicles = pd.read_csv('../data/crashes_vehicles.csv')

### Dealing with NANs

In [ ]:
nan_count = vehicles.isna().sum()

useful_columns = nan_count[nan_count < 1_000_000].index

vehicles = vehicles[useful_columns]

vehicles['VEHICLE_DEFECT'].replace({np.nan: 'UNKNOWN', }, inplace = True)

vehicles['VEHICLE_TYPE'].replace({np.nan: 'UNKNOWN/NA'}, inplace = True)

vehicles['VEHICLE_USE'].replace({np.nan: 'UNKNOWN/NA'}, inplace = True)

vehicles['MANEUVER'].replace({np.nan: 'UNKNOWN/NA'}, inplace = True)

vehicles['OCCUPANT_CNT'].replace({np.nan: 999}, inplace = True)

vehicles['FIRST_CONTACT_POINT'].replace({np.nan: 'UNKOWN' }, inplace = True)



### Converting to datetime

In [ ]:
vehicles['CRASH_DATE'] = pd.to_datetime(vehicles['CRASH_DATE'])

### Categorical -> Numeric

In [ ]:
cat_columns = ['UNIT_TYPE', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'MANEUVER',\
              'FIRST_CONTACT_POINT']

vehicles_cleaned = pd.get_dummies(vehicles, columns = cat_columns, drop_first = True, dtype = int)